# English Premier League and the Ratings Percentage Index

This notebook uses python tools to automatically generate the English Premier League table ordered by the Ratings Percetage Index (RPI), using football match results data at [www.football-data.co.uk](http://www.football-data.co.uk/englandm.php). The RPI is a technique proposed by [The Tomkins Times](https://tomkinstimes.com/) subscriber Tim O'Brien to take account of the quality of opposition and is described [here](https://tomkinstimes.com/2016/11/comment-of-the-month-october-2016/). The solution is built into a simple web app.

The project uses [jupyter notebook](http://jupyter.org/index.html), [python](https://www.python.org/), [pandas](http://pandas.pydata.org/), [beautiful soup](https://www.crummy.com/software/BeautifulSoup/), [requests](http://docs.python-requests.org/en/master/), [spyre](https://github.com/adamhajari/spyre) and [heroku](https://www.heroku.com/).

#### Notebook Change Log

In [17]:
%%html
<! left align the change log table in next cell >
<style>
table {float:left}
</style>

| Date          | Change Description |
| :------------ | :----------------- |
| 17th November 2016 | Initial baseline |
| 30th October 2017 | Updated *gen_prem_table_RPI()* to fix error in calculation for P (number of games played) |
| 5th November 2017 | Updated *gen_prem_table_RPI()* so that table is generated for 1st week |


## Set-up

Import the modules needed for the analysis.

In [18]:
import pandas as pd
import sys 
import requests
import datetime as dt
import pickle
import os
import bs4
from bs4 import BeautifulSoup, SoupStrainer
from itertools import cycle
from collections import defaultdict
from IPython.display import Image
from IPython.core.display import HTML 
from __future__ import division

# enable inline plotting
#%matplotlib inline

Print version numbers of key modules.

In [19]:
print 'python version: {}'.format(sys.version)
print 'pandas version: {}'.format(pd.__version__)
print 'requests version: {}'.format(requests.__version__)
print 'bs4 version: {}'.format(bs4.__version__)
#print 'matplotlib version: {}'.format(mpl.__version__)
#print 'numpy version: {}'.format(np.__version__)

python version: 2.7.11 |Anaconda custom (64-bit)| (default, Feb 16 2016, 09:58:36) [MSC v.1500 64 bit (AMD64)]
pandas version: 0.18.0
requests version: 2.13.0
bs4 version: 4.4.1


## Generate the Premier League table with RPI

Start by defining some utility functions

In [20]:
def get_pl_master_data():
    """Return url of latest premier league results file and the date the file was last updated.
    
    Data source is www.football-data.co.uk.
    Format of returned date is "%Y-%m-%d" (the pandas default).
    """
    # scrape the data from football-data website
    URL_FD_ROOT = 'http://www.football-data.co.uk/'
    ENGLAND_LOCATION = 'englandm.php'
    PL_TEXT = 'Premier League'
    with requests.Session() as session:
        response = session.get(URL_FD_ROOT + ENGLAND_LOCATION)
        soup = BeautifulSoup(response.content, 'lxml')

        # scrape last updated date
        last_updated_tag = soup.find_all('i')[0]
        last_updated_date = last_updated_tag.text.split('Last updated: \t')[1]
        # set date format to be same as pandas default
        last_updated_date = dt.datetime.strptime(last_updated_date, '%d/%m/%y').strftime('%Y-%m-%d')
                                                                                         
        # scrape url of premier league results file
        latest_pl_results_file_tag = soup.findAll('a', href=True, text=PL_TEXT)[0]['href']
        url_latest_pl_results_file = URL_FD_ROOT + latest_pl_results_file_tag                                                                         
                                                                                     
    return(url_latest_pl_results_file, last_updated_date)

In [21]:
# check current latest
url_latest_pl_results_file, last_updated_date = get_pl_master_data()
print 'PL results URL: {}, last updated: {}'.format(url_latest_pl_results_file,  last_updated_date)

PL results URL: http://www.football-data.co.uk/mmz4281/1718/E0.csv, last updated: 2017-11-01


In [22]:
def get_pl_results_dataframe(update_cache=False):
    """Return latest premier league results as a dataframe and the date of the results data.
    
    Data source is www.football-data.co.uk.
    Cache data locally to avoid unnecessary calls to football-data website.
    Download results from master data source if local data is out of date.
    """
    
    LOCAL_RESULTS_DATA_FILE = 'data/E0.csv'
    PICKLE_FILE = 'save.p' # holds date of results data file

    # get master data source data
    url_latest_pl_results_file, master_results_data_date = get_pl_master_data()
    
    if update_cache:
        if os.path.exists(PICKLE_FILE):
            os.remove(PICKLE_FILE)

    # get local data source date
    if os.path.exists(PICKLE_FILE):
        local_results_data_date = pickle.load(open(PICKLE_FILE, 'rb'))
    else:
        local_results_data_date = None

    if local_results_data_date < master_results_data_date:
        print 'local results data out of date, updating from master results data file'
        parse_dates_col = ['Date']
        df_results = pd.read_csv(url_latest_pl_results_file, parse_dates=parse_dates_col, dayfirst=True)
        df_results.to_csv(LOCAL_RESULTS_DATA_FILE, index=False)
        local_results_data_date = master_results_data_date
        pickle.dump(local_results_data_date, open(PICKLE_FILE, 'wb'))
    else:
        print 'local results data still latest'
        parse_dates_col = ['Date']
        df_results = pd.read_csv(LOCAL_RESULTS_DATA_FILE, parse_dates=parse_dates_col, dayfirst=True)
        
    return df_results, local_results_data_date

In [23]:
# check results
df_results, results_date = get_pl_results_dataframe()
print 'results data date: {}'.format(results_date)
print df_results.dtypes.head()
df_results.head()

local results data still latest
results data date: 2017-11-01
Div                 object
Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG                 int64
dtype: object


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,2017-08-11,Arsenal,Leicester,4,3,H,2,2,D,...,2.32,21,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25
1,E0,2017-08-12,Brighton,Man City,0,2,A,0,0,D,...,2.27,20,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29
2,E0,2017-08-12,Chelsea,Burnley,2,3,A,0,3,A,...,2.23,20,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25
3,E0,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,1.72,18,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51
4,E0,2017-08-12,Everton,Stoke,1,0,H,1,0,H,...,1.76,19,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42


In [24]:
# check forced update to cache
df_results, results_date = get_pl_results_dataframe(update_cache=True)
print 'results data date: {}'.format(results_date)
print df_results.dtypes.head()
df_results.head()

local results data out of date, updating from master results data file
results data date: 2017-11-01
Div                 object
Date        datetime64[ns]
HomeTeam            object
AwayTeam            object
FTHG                 int64
dtype: object


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,2017-08-11,Arsenal,Leicester,4,3,H,2,2,D,...,2.32,21,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25
1,E0,2017-08-12,Brighton,Man City,0,2,A,0,0,D,...,2.27,20,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29
2,E0,2017-08-12,Chelsea,Burnley,2,3,A,0,3,A,...,2.23,20,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25
3,E0,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,1.72,18,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51
4,E0,2017-08-12,Everton,Stoke,1,0,H,1,0,H,...,1.76,19,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42


In [25]:
# check forced reload
local_data_source_date = '2016-11-05'
PICKLE_FILE = 'save.p'
pickle.dump(local_data_source_date, open(PICKLE_FILE, 'wb'))
print pickle.load(open(PICKLE_FILE, 'rb'))
df_results, results_date = get_pl_results_dataframe()
print 'results data date: {}'.format(results_date)
df_results.head()

2016-11-05
local results data out of date, updating from master results data file
results data date: 2017-11-01


,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
0,E0,2017-08-11,Arsenal,Leicester,4,3,H,2,2,D,...,2.32,21,-1.00,1.91,1.85,2.10,2.02,1.49,4.73,7.25
1,E0,2017-08-12,Brighton,Man City,0,2,A,0,0,D,...,2.27,20,1.50,1.95,1.91,2.01,1.96,11.75,6.15,1.29
2,E0,2017-08-12,Chelsea,Burnley,2,3,A,0,3,A,...,2.23,20,-1.75,2.03,1.97,1.95,1.90,1.33,5.40,12.25
3,E0,2017-08-12,Crystal Palace,Huddersfield,0,3,A,0,2,A,...,1.72,18,-0.75,2.10,2.05,1.86,1.83,1.79,3.56,5.51
4,E0,2017-08-12,Everton,Stoke,1,0,H,1,0,H,...,1.76,19,-0.75,1.94,1.90,2.01,1.98,1.82,3.49,5.42


In [26]:
df_results.tail()

,Div,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTHG,HTAG,HTR,...,BbAv<2.5,BbAH,BbAHh,BbMxAHH,BbAvAHH,BbMxAHA,BbAvAHA,PSCH,PSCD,PSCA
95,E0,2017-10-28,Watford,Stoke,0,1,A,0,1,A,...,1.91,19,-0.25,1.78,1.74,2.22,2.16,2.02,3.57,4.07
96,E0,2017-10-28,West Brom,Man City,2,3,A,1,2,A,...,2.48,19,1.75,1.98,1.93,1.98,1.94,11.00,6.40,1.29
97,E0,2017-10-29,Brighton,Southampton,1,1,D,0,1,A,...,1.54,18,0.25,1.88,1.83,2.11,2.06,3.42,3.29,2.36
98,E0,2017-10-29,Leicester,Everton,2,0,H,2,0,H,...,1.86,19,-0.25,1.88,1.84,2.11,2.04,2.08,3.45,4.00
99,E0,2017-10-30,Burnley,Newcastle,1,0,H,0,0,D,...,1.60,18,0.00,2.11,2.04,1.85,1.83,3.22,3.13,2.56


In [27]:
def validate_date(date_text):
    """Raise error if date format is not YYYY-MM-DD."""
    try:
        dt.datetime.strptime(date_text, '%Y-%m-%d')
    except ValueError:
        raise ValueError("Incorrect date format, should be YYYY-MM-DD")

In [28]:
validate_date('2016-11-14')
#validate_date('14-11-2016') # produces error

In [29]:
def simple_date(date_text):
    """Return given date in format '%y-%m-%d' to '%d %b %y'."""
    validate_date(date_text)
    return (dt.datetime.strptime(date_text, '%Y-%m-%d').strftime('%d %b %y'))

In [30]:
print simple_date('2016-6-2')
print simple_date('2017-1-31')

02 Jun 16
31 Jan 17


Now produce function to generate the Prem League Table with RPI

Function Change Log
- October 2017, Updated gen_prem_table_RPI() to fix error in calculation for P (number of games played)
is listed but not yet played, so game data is not available
- November 2017, Updated gen_prem_table_RPI() so that table is generated for 1st week

In [31]:
def gen_prem_table_RPI(before_date=None, update_cache=False):
    """Return prem table with RPI at given before_date and return data source date."""
    
    results = []
    opponents_d = {}
    df_results, results_date = get_pl_results_dataframe(update_cache)
    
    # filter results in dataframe at given before_date
    if before_date:
        validate_date(before_date)
        df_results = df_results[df_results.Date <= before_date]
        
    for team in set(df_results.HomeTeam.tolist() + df_results.AwayTeam.tolist()):
        home_results = df_results[df_results['HomeTeam'] == team]
        home_played = len(home_results.index)
        home_win = home_results.FTR[home_results.FTR == 'H'].count()
        home_draw = home_results.FTR[home_results.FTR == 'D'].count()
        home_lose = home_results.FTR[home_results.FTR == 'A'].count()
        home_goals_for = home_results.FTHG.sum()
        home_goals_against = home_results.FTAG.sum()
        home_opponents = list(df_results[df_results.HomeTeam == team].AwayTeam.values)

        away_results = df_results[df_results['AwayTeam'] == team]
        away_played = len(away_results.index)
        away_win = away_results.FTR[away_results.FTR == 'A'].count()
        away_draw = away_results.FTR[away_results.FTR == 'D'].count()
        away_lose = away_results.FTR[away_results.FTR == 'H'].count()
        away_goals_for = away_results.FTAG.sum()
        away_goals_against = away_results.FTHG.sum()
        away_opponents = list(df_results[df_results.AwayTeam == team].HomeTeam.values)

        # add team opponents to dictionary
        team_opponents = home_opponents + away_opponents
        opponents_d[team] = team_opponents
        
        # create team results dictionary and add to results list
        result_d = {} 
        result_d['Team'] = team
        result_d['W'] = home_win + away_win
        result_d['D'] = home_draw + away_draw
        result_d['L'] = home_lose + away_lose
        result_d['GF'] = home_goals_for + away_goals_for
        result_d['GA'] = home_goals_against + away_goals_against
        result_d['GD'] = result_d['GF'] - result_d['GA']
        result_d['PTS'] = result_d['W']*3 + result_d['D']
        result_d['P'] = result_d['W'] + result_d['D'] + result_d['L']
        results.append(result_d) # append team result dictionary to list of results

    # create PL table dataframe from team results and sort by points (and then goal difference and goals for)
    # show date of data in Position column
    PLtable = pd.DataFrame(results, columns=['Team', 'P', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS'])
    PLtable.sort_values(['PTS', 'GD', 'GF'], ascending=False, inplace=True)
    col_date = before_date if before_date else results_date
    pos_title = 'Position at {}'.format(simple_date(col_date))
    PLtable[pos_title] = range(1, len(PLtable)+1) # add new column for position, with highest points first
    PLtable.set_index([pos_title], inplace=True, drop=True) 
    #PLtable.reset_index(inplace=True)
    
    # Add RPI to the table
    PLtable['PTS%'] = 100*(PLtable.PTS/(PLtable.P*3))
    PLtable['OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['PTS%'].mean(), axis=1)
    PLtable['OPP_OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['OPP_PTS%'].mean(), axis=1)
    PLtable['RPI'] = (PLtable['PTS%']*.25 + PLtable['OPP_PTS%']*.50 + PLtable['OPP_OPP_PTS%']*.25)
    # replace nan with 0 just in case results are published when all games haven't yet been played 
    PLtable.fillna(0, inplace=True)  
    PLtable['RPI_Position'] = PLtable['RPI'].rank(ascending=False).astype(int)
    
    # return PL table with RPI, sorted by RPI and PTS percentage
    return(PLtable.sort_values(['RPI', 'PTS%'], ascending=False), results_date)

In [32]:
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI()
print 'Premier league table ordered by RPI, date of results data: {}'.format(results_date)
PLtableRPI

local results data still latest
Premier league table ordered by RPI, date of results data: 2017-11-01


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 17,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35,6,29,28,93.3,39.3,49.9,55.5,1
6,Liverpool,10,4,4,2,17,16,1,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18,10,8,19,63.3,46.7,47.3,51.0,3
3,Tottenham,10,6,2,2,19,7,12,20,66.7,44.0,46.9,50.4,4
2,Man United,10,7,2,1,23,4,19,23,76.7,37.7,47.9,50.0,5
7,Burnley,10,4,4,2,9,9,0,16,53.3,46.7,46.9,48.4,6
8,Watford,10,4,3,3,15,18,-3,15,50.0,47.7,46.2,47.9,7
5,Arsenal,10,6,1,3,19,13,6,19,63.3,39.3,49.2,47.8,8
14,Stoke,10,3,2,5,11,20,-9,11,36.7,52.0,44.5,46.3,9


In [33]:
pd.set_option('precision', 1)
request_date = dt.datetime.today().strftime("%Y-%m-%d")
PLtableRPI, results_date = gen_prem_table_RPI(before_date=request_date, update_cache=False)
print 'Premier league table ordered by RPI at {}, date of results data: {}'.format(request_date, results_date)
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2017-11-05, date of results data: 2017-11-01


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 05 Nov 17,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35,6,29,28,93.3,39.3,49.9,55.5,1
6,Liverpool,10,4,4,2,17,16,1,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18,10,8,19,63.3,46.7,47.3,51.0,3
3,Tottenham,10,6,2,2,19,7,12,20,66.7,44.0,46.9,50.4,4
2,Man United,10,7,2,1,23,4,19,23,76.7,37.7,47.9,50.0,5
7,Burnley,10,4,4,2,9,9,0,16,53.3,46.7,46.9,48.4,6
8,Watford,10,4,3,3,15,18,-3,15,50.0,47.7,46.2,47.9,7
5,Arsenal,10,6,1,3,19,13,6,19,63.3,39.3,49.2,47.8,8
14,Stoke,10,3,2,5,11,20,-9,11,36.7,52.0,44.5,46.3,9


In [34]:
pd.set_option('precision', 1)
request_date = '2017-10-24' # 9 games 2017
PLtableRPI, results_date = gen_prem_table_RPI(request_date)
print 'Premier league table ordered by RPI at {}, date of results data: {}'.format(request_date, results_date)
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2017-10-24, date of results data: 2017-11-01


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 24 Oct 17,,,,,,,,,,,,,,
1,Man City,9,8,1,0,32,4,28,25,92.6,38.7,51.0,55.2,1
3,Tottenham,9,6,2,1,19,6,13,20,74.1,40.7,48.5,51.0,2
4,Chelsea,9,5,1,3,17,10,7,16,59.3,48.1,47.2,50.7,3
9,Liverpool,9,3,4,2,14,16,-2,13,48.1,55.6,43.2,50.6,4
6,Watford,9,4,3,2,15,17,-2,15,55.6,48.6,45.1,49.5,5
8,Burnley,9,3,4,2,8,9,-1,13,48.1,47.3,47.6,47.6,6
2,Man United,9,6,2,1,22,4,18,20,74.1,33.3,49.5,47.6,7
5,Arsenal,9,5,1,3,17,12,5,16,59.3,39.9,49.4,47.1,8
18,Everton,9,2,2,5,7,18,-11,8,29.6,56.0,44.1,46.4,9


In [35]:
pd.set_option('precision', 1)
request_date = '2017-10-31' # 10 games 2017
PLtableRPI, results_date = gen_prem_table_RPI(request_date)
print 'Premier league table ordered by RPI at {}, date of results data: {}'.format(request_date, results_date)
PLtableRPI

local results data still latest
Premier league table ordered by RPI at 2017-10-31, date of results data: 2017-11-01


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 31 Oct 17,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35,6,29,28,93.3,39.3,49.9,55.5,1
6,Liverpool,10,4,4,2,17,16,1,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18,10,8,19,63.3,46.7,47.3,51.0,3
3,Tottenham,10,6,2,2,19,7,12,20,66.7,44.0,46.9,50.4,4
2,Man United,10,7,2,1,23,4,19,23,76.7,37.7,47.9,50.0,5
7,Burnley,10,4,4,2,9,9,0,16,53.3,46.7,46.9,48.4,6
8,Watford,10,4,3,3,15,18,-3,15,50.0,47.7,46.2,47.9,7
5,Arsenal,10,6,1,3,19,13,6,19,63.3,39.3,49.2,47.8,8
14,Stoke,10,3,2,5,11,20,-9,11,36.7,52.0,44.5,46.3,9


## Building The Spyre App

Useful reference material:
+ How to develop a Spyre app, including tutorials - [https://github.com/adamhajari/spyre](https://github.com/adamhajari/spyre).

Spyre is a web app framework for providing a simple user interface for Python data projects. In simple terms the premrpi app involves:
1. creating a user interface to call the getData() function to return a data table.
2. using pandas (and related modules) to generate the data table.

See [premrpi_app.py in the premrpi github repo](https://github.com/terrydolan/premrpi) for the app source code.

---

## Edge Case Testing

Test at 2017 week 1, week 1 with partial data, week 2 and week 3

In [36]:
' gen_prem_table_RPI amended to parse an input test file'
def gen_prem_table_RPI_test(before_date=None, update_cache=False, test_file='data/E0.csv'):
    """Return prem table with RPI at given before_date and return data source date."""
    
    results = []
    opponents_d = {}
    
    # generate results dataframe from test_file if provided
    if not test_file:
        df_results, results_date = get_pl_results_dataframe(update_cache)
    else: 
        # testing - read results from test file and set results_date to that in pickle file (save.p)
        print '*** Testing with {} ***'.format(test_file)
        df_results = pd.read_csv(test_file, parse_dates=['Date'], dayfirst=True)
        results_date = local_results_data_date = pickle.load(open('save.p', 'rb'))
    
    # filter results in dataframe at given before_date
    if before_date:
        validate_date(before_date)
        df_results = df_results[df_results.Date <= before_date]
    
    for team in set(df_results.HomeTeam.tolist() + df_results.AwayTeam.tolist()):
        home_results = df_results[df_results['HomeTeam'] == team]
        home_played = len(home_results.index)
        home_win = home_results.FTR[home_results.FTR == 'H'].count()
        home_draw = home_results.FTR[home_results.FTR == 'D'].count()
        home_lose = home_results.FTR[home_results.FTR == 'A'].count()
        home_goals_for = home_results.FTHG.sum()
        home_goals_against = home_results.FTAG.sum()
        home_opponents = list(df_results[df_results.HomeTeam == team].AwayTeam.values)

        away_results = df_results[df_results['AwayTeam'] == team]
        away_played = len(away_results.index)
        away_win = away_results.FTR[away_results.FTR == 'A'].count()
        away_draw = away_results.FTR[away_results.FTR == 'D'].count()
        away_lose = away_results.FTR[away_results.FTR == 'H'].count()
        away_goals_for = away_results.FTAG.sum()
        away_goals_against = away_results.FTHG.sum()
        away_opponents = list(df_results[df_results.AwayTeam == team].HomeTeam.values)

        # add team opponents to dictionary
        team_opponents = home_opponents + away_opponents
        opponents_d[team] = team_opponents
        
        # create team results dictionary and add to results list
        result_d = {} 
        result_d['Team'] = team
        result_d['W'] = home_win + away_win
        result_d['D'] = home_draw + away_draw
        result_d['L'] = home_lose + away_lose
        result_d['GF'] = home_goals_for + away_goals_for
        result_d['GA'] = home_goals_against + away_goals_against
        result_d['GD'] = result_d['GF'] - result_d['GA']
        result_d['PTS'] = result_d['W']*3 + result_d['D']
        result_d['P'] = result_d['W'] + result_d['D'] + result_d['L']
        results.append(result_d) # append team result dictionary to list of results

    # create PL table dataframe from team results and sort by points (and then goal difference and goals for)
    # show date of data in Position column
    PLtable = pd.DataFrame(results, columns=['Team', 'P', 'W', 'D', 'L', 'GF', 'GA', 'GD', 'PTS'])
    PLtable.sort_values(['PTS', 'GD', 'GF'], ascending=False, inplace=True)
    col_date = before_date if before_date else results_date
    pos_title = 'Position at {}'.format(simple_date(col_date))
    PLtable[pos_title] = range(1, len(PLtable)+1) # add new column for position, with highest points first
    PLtable.set_index([pos_title], inplace=True, drop=True) 
    #PLtable.reset_index(inplace=True)
    
    # Add RPI to the table
    PLtable['PTS%'] = 100*(PLtable.PTS/(PLtable.P*3))
    PLtable['OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['PTS%'].mean(), axis=1)
    PLtable['OPP_OPP_PTS%'] = PLtable.apply(lambda row: PLtable[PLtable.Team.isin(opponents_d[row.Team])]['OPP_PTS%'].mean(), axis=1)
    PLtable['RPI'] = (PLtable['PTS%']*.25 + PLtable['OPP_PTS%']*.50 + PLtable['OPP_OPP_PTS%']*.25)
    PLtable.fillna(0, inplace=True)
    PLtable['RPI_Position'] = PLtable['RPI'].rank(ascending=False).astype(int)
    
    # return PL table with RPI, sorted by RPI and PTS percentage
    return(PLtable.sort_values(['RPI', 'PTS%'], ascending=False), results_date)

In [37]:
# Test using 2017
# expect all teams to have played 1 game
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk1_part.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk1_part.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 17,,,,,,,,,,,,,,
1,Man United,1,1,0,0,4.0,0.0,4.0,3,100.0,0.0,100.0,50.0,7
2,Huddersfield,1,1,0,0,3.0,0.0,3.0,3,100.0,0.0,100.0,50.0,7
3,Man City,1,1,0,0,2.0,0.0,2.0,3,100.0,0.0,100.0,50.0,7
4,Arsenal,1,1,0,0,4.0,3.0,1.0,3,100.0,0.0,100.0,50.0,7
5,Burnley,1,1,0,0,3.0,2.0,1.0,3,100.0,0.0,100.0,50.0,7
6,Everton,1,1,0,0,1.0,0.0,1.0,3,100.0,0.0,100.0,50.0,7
7,West Brom,1,1,0,0,1.0,0.0,1.0,3,100.0,0.0,100.0,50.0,7
14,Leicester,1,0,0,1,3.0,4.0,-1.0,0,0.0,100.0,0.0,50.0,7
15,Chelsea,1,0,0,1,2.0,3.0,-1.0,0,0.0,100.0,0.0,50.0,7


In [38]:
# Test using 2017
# expect all teams to have played 1 game, with exception of 
# Newcastle vs Southampton which has not yet been played
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk1_part.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk1_part.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 17,,,,,,,,,,,,,,
1,Man United,1,1,0,0,4.0,0.0,4.0,3,100.0,0.0,100.0,50.0,7
2,Huddersfield,1,1,0,0,3.0,0.0,3.0,3,100.0,0.0,100.0,50.0,7
3,Man City,1,1,0,0,2.0,0.0,2.0,3,100.0,0.0,100.0,50.0,7
4,Arsenal,1,1,0,0,4.0,3.0,1.0,3,100.0,0.0,100.0,50.0,7
5,Burnley,1,1,0,0,3.0,2.0,1.0,3,100.0,0.0,100.0,50.0,7
6,Everton,1,1,0,0,1.0,0.0,1.0,3,100.0,0.0,100.0,50.0,7
7,West Brom,1,1,0,0,1.0,0.0,1.0,3,100.0,0.0,100.0,50.0,7
14,Leicester,1,0,0,1,3.0,4.0,-1.0,0,0.0,100.0,0.0,50.0,7
15,Chelsea,1,0,0,1,2.0,3.0,-1.0,0,0.0,100.0,0.0,50.0,7


In [39]:
# Test using 2017
# expect all teams to have played 2 games
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk2.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk2.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 17,,,,,,,,,,,,,,
13,Burnley,2,1,0,1,3,3,0,3,50.0,75.0,37.5,59.4,1
3,West Brom,2,2,0,0,2,0,2,6,100.0,25.0,79.2,57.3,2
8,Everton,2,1,1,0,2,1,1,4,66.7,58.3,45.8,57.3,3
14,Stoke,2,1,0,1,1,1,0,3,50.0,58.3,54.2,55.2,4
1,Man United,2,2,0,0,8,0,8,6,100.0,8.3,83.3,50.0,5
12,Chelsea,2,1,0,1,4,4,0,3,50.0,50.0,50.0,50.0,5
16,Bournemouth,2,0,0,2,0,3,-3,0,0.0,83.3,29.2,49.0,7
11,Arsenal,2,1,0,1,4,4,0,3,50.0,50.0,41.7,47.9,8
15,Swansea,2,0,1,1,0,4,-4,1,16.7,83.3,8.3,47.9,8


In [40]:
# Test using 2017
# expect all teams to have played 3 games
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk3.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk3.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 17,,,,,,,,,,,,,,
6,Chelsea,3,2,0,1,6,4,2,6,66.7,44.4,58.0,53.4,1
10,Burnley,3,1,1,1,4,4,0,4,44.4,63.0,40.7,52.8,2
12,Everton,3,1,1,1,2,3,-1,4,44.4,63.0,38.3,52.2,3
1,Man United,3,3,0,0,10,0,10,9,100.0,25.9,54.3,51.5,4
5,West Brom,3,2,1,0,3,1,2,7,77.8,29.6,61.7,49.7,5
11,Stoke,3,1,1,1,2,2,0,4,44.4,51.9,48.1,49.1,6
13,Swansea,3,1,1,1,2,4,-2,4,44.4,51.9,44.4,48.1,7
9,Tottenham,3,1,1,1,4,3,1,4,44.4,48.1,49.4,47.5,8
2,Liverpool,3,2,1,0,8,3,5,7,77.8,29.6,49.4,46.6,9


In [41]:
# Test using 2017
# expect all teams to have played 10 games
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk10.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk10.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 17,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35,6,29,28,93.3,39.3,49.9,55.5,1
6,Liverpool,10,4,4,2,17,16,1,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18,10,8,19,63.3,46.7,47.3,51.0,3
3,Tottenham,10,6,2,2,19,7,12,20,66.7,44.0,46.9,50.4,4
2,Man United,10,7,2,1,23,4,19,23,76.7,37.7,47.9,50.0,5
7,Burnley,10,4,4,2,9,9,0,16,53.3,46.7,46.9,48.4,6
8,Watford,10,4,3,3,15,18,-3,15,50.0,47.7,46.2,47.9,7
5,Arsenal,10,6,1,3,19,13,6,19,63.3,39.3,49.2,47.8,8
14,Stoke,10,3,2,5,11,20,-9,11,36.7,52.0,44.5,46.3,9


In [42]:
# Test using 2017
# expect all teams to have played 10 games with exception of Burnley and Newcastle who have played 9
# Burnley vs Newcastle not included as played on Monday 30th October
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2017-18_wk10_part.csv')
PLtableRPI

*** Testing with data/E0_2017-18_wk10_part.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 17,,,,,,,,,,,,,,
1,Man City,10,9,1,0,35.0,6.0,29.0,28,93.3,38.8,49.9,55.2,1
6,Liverpool,10,4,4,2,17.0,16.0,1.0,16,53.3,54.3,43.7,51.4,2
4,Chelsea,10,6,1,3,18.0,10.0,8.0,19,63.3,46.1,47.3,50.7,3
3,Tottenham,10,6,2,2,19.0,7.0,12.0,20,66.7,44.0,46.8,50.4,4
2,Man United,10,7,2,1,23.0,4.0,19.0,23,76.7,37.7,48.0,50.0,5
7,Watford,10,4,3,3,15.0,18.0,-3.0,15,50.0,47.7,46.2,47.9,6
5,Arsenal,10,6,1,3,19.0,13.0,6.0,19,63.3,39.3,49.2,47.8,7
10,Burnley,9,3,4,2,8.0,9.0,-1.0,13,48.1,47.2,46.7,47.3,8
14,Stoke,10,3,2,5,11.0,20.0,-9.0,11,36.7,52.5,44.3,46.5,9


In [43]:
#E0_2016-17_final.csv

In [44]:
# Test using 2016
# expect all teams to have played 38 games
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(test_file='data/E0_2016-17_final.csv')
PLtableRPI

*** Testing with data/E0_2016-17_final.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 17,,,,,,,,,,,,,,
1,Chelsea,38,30,3,5,85,33,52,93,81.6,44.5,46.4,54.2,1
2,Tottenham,38,26,8,4,86,26,60,86,75.4,44.8,46.4,52.9,2
3,Man City,38,23,9,6,80,39,41,78,68.4,45.2,46.4,51.3,3
4,Liverpool,38,22,10,6,78,42,36,76,66.7,45.2,46.4,50.9,4
5,Arsenal,38,23,6,9,77,44,33,75,65.8,45.3,46.4,50.7,5
6,Man United,38,18,15,5,54,29,25,69,60.5,45.6,46.4,49.5,6
7,Everton,38,17,10,11,62,44,18,61,53.5,45.9,46.3,47.9,7
8,Southampton,38,12,10,16,41,48,-7,46,40.4,46.6,46.3,45.0,8
9,Bournemouth,38,12,10,16,55,67,-12,46,40.4,46.6,46.3,45.0,8


In [45]:
# Test using 2016
# expect all teams to have played 9 games - see expected results from TTT below
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(before_date='2016-10-25', test_file='data/E0_2016-17_final.csv')
PLtableRPI

*** Testing with data/E0_2016-17_final.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 25 Oct 16,,,,,,,,,,,,,,
3,Liverpool,9,6,2,1,20,11,9,20,74.1,47.3,47.6,54.1,1
4,Chelsea,9,6,1,2,19,9,10,19,70.4,44.9,48.1,52.0,2
2,Arsenal,9,6,2,1,19,9,10,20,74.1,42.8,47.5,51.8,3
1,Man City,9,6,2,1,20,9,11,20,74.1,40.7,46.1,50.4,4
5,Tottenham,9,5,4,0,13,4,9,19,70.4,43.6,43.9,50.4,5
7,Man United,9,4,2,3,13,12,1,14,51.9,50.6,45.7,49.7,6
10,Bournemouth,9,3,3,3,12,12,0,12,44.4,48.6,44.1,46.4,7
12,Leicester,9,3,2,4,11,15,-4,11,40.7,49.0,46.8,46.4,8
14,Burnley,9,3,1,5,8,13,-5,10,37.0,50.2,46.3,45.9,9


In [46]:
# Test using 2016
# expect all teams to have played 10 games - see expected results from TTT below
pd.set_option('precision', 1)
PLtableRPI, results_date = gen_prem_table_RPI_test(before_date='2016-11-1', test_file='data/E0_2016-17_final.csv')
PLtableRPI

*** Testing with data/E0_2016-17_final.csv ***


,Team,P,W,D,L,GF,GA,GD,PTS,PTS%,OPP_PTS%,OPP_OPP_PTS%,RPI,RPI_Position
Position at 01 Nov 16,,,,,,,,,,,,,,
3,Liverpool,10,7,2,1,24,13,11,23,76.7,45.3,47.4,53.7,1
4,Chelsea,10,7,1,2,21,9,12,22,73.3,44.7,47.4,52.5,2
2,Arsenal,10,7,2,1,23,10,13,23,76.7,40.0,47.8,51.1,3
1,Man City,10,7,2,1,24,9,15,23,76.7,39.0,46.3,50.2,4
5,Tottenham,10,5,5,0,14,5,9,20,66.7,44.3,44.6,50.0,5
8,Man United,10,4,3,3,13,12,1,15,50.0,50.0,45.5,48.9,6
11,Leicester,10,3,3,4,12,16,-4,12,40.0,50.0,46.8,46.7,7
6,Everton,10,5,3,2,15,8,7,18,60.0,40.3,45.1,46.4,8
14,Burnley,10,3,2,5,8,13,-5,11,36.7,51.0,46.3,46.2,9


In [47]:
#Note: results after 10 games in 2016...
Image(url= "https://tomkinstimes.com/wp-content/uploads/2016/11/RPI-by-Tim-OBrien.png")